<a href="https://colab.research.google.com/github/vjrdy/Machine-Language-Translation/blob/master/Machine_Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a translation system from a parallel corpus is a resource-intensive process (processor, memory, GPU).

***First Go to Runtime and  change the runtime type to GPU.***


<br>
 Copyright:  Vijay YERUVA 
<br>
 Email: vijay.yeruva@epita.fr
<br>
 UID: #20527











# Git Clone
First Git clone the OpenNMT source

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls gdrive/My\ Drive/model_bckp/run/

assets
checkpoint
eval
events.out.tfevents.1563743941.b5886273c130
events.out.tfevents.1563744101.b5886273c130
export
graph.pbtxt
model.ckpt-12000.data-00000-of-00002
model.ckpt-12000.data-00001-of-00002
model.ckpt-12000.index
model.ckpt-12000.meta
model.ckpt-13000.data-00000-of-00002
model.ckpt-13000.data-00001-of-00002
model.ckpt-13000.index
model.ckpt-13000.meta
model.ckpt-14000.data-00000-of-00002
model.ckpt-14000.data-00001-of-00002
model.ckpt-14000.index
model.ckpt-14000.meta
model.ckpt-15000.data-00000-of-00002
model.ckpt-15000.data-00001-of-00002
model.ckpt-15000.index
model.ckpt-15000.meta
model.ckpt-16000.data-00000-of-00002
model.ckpt-16000.data-00001-of-00002
model.ckpt-16000.index
model.ckpt-16000.meta
model_description.py
projector_config.pbtxt
seq2seq_decoder_w_embs.txt
seq2seq_encoder_w_embs.txt


In [2]:
!git clone https://github.com/OpenNMT/OpenNMT-tf.git

Cloning into 'OpenNMT-tf'...
remote: Enumerating objects: 1284, done.
remote: Counting objects: 100% (1284/1284), done.
remote: Compressing objects: 100% (485/485), done.
remote: Total 16561 (delta 857), reused 1171 (delta 791), pack-reused 15277
Receiving objects: 100% (16561/16561), 14.73 MiB | 25.43 MiB/s, done.
Resolving deltas: 100% (13274/13274), done.


In [3]:
!ls

gdrive	OpenNMT-tf  sample_data


In [4]:
pip install OpenNMT-tf[tensorflow_gpu]

     |████████████████████████████████| 153kB 3.4MB/s 
     |████████████████████████████████| 276kB 44.7MB/s 
     |████████████████████████████████| 2.2MB 48.3MB/s 
     |████████████████████████████████| 377.0MB 83kB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
  Stored in directory: /root/.cache/pip/wheels/25/26/75/6598d35c0498433bbfb118f37bf85da0eb5dc672836d025553
Successfully built pyyaml sacrebleu
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Please install  OpenNMT-tensorflow use by pip



In [0]:
!pip install tensorflow-gpu==1.14

# Theory explanation

**Machine translation is a field of natural language processing, meaning that computers translate one language into another.**

Rule based, and statistical based, and recently we are using Deep Learning-based machine translation.

Learn how to build a real machine translation system and how the system pipeline is structured. Most of these courses can be applied to basic natural language processing problems as well as machine translation.

**Step**



**1.   Data Collection**

Parallel corpus is collected from various sources. It is possible to collect news texts, drama / movie subtitles, Wikipedia, etc., as well as data sets for evaluation of translation systems disclosed by WMT, a machine translation competition, and use them in translation systems.


**2.   Cleaning**

The collected data must be refined. The refinement process includes sorting sentences by corpus in both languages, and eliminating noise such as special characters.


**3. Subword Tokenization**

Refine spacing using the POS tagger or segmenter for each language. English may have refinement issues in upper / lower case.
After the spacing is refined, use Byte Pair Encoding (BPE) using public tools such as Subword or WordPiece. This allows you to perform additional segments and construct a vocabulary list. At this time, the segmented models learned for the BPE segment should be kept for future use.


**4. Train**

Train the seq2seq model using prepared datasets. Depending on the amount, you can train with a single GPU, or use multiple GPUs in parallel to reduce training time.


**5. Translate**

Now that the model has been created, you can start translating.


**6. Detokenization**

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.


**7. Evaluating**

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

# Data Collection

Let's Collect en-de Parallel Corpus form amazon S3
In your Colab Files A directory called toy-ende would have been created.

In [5]:
!wget https://cservan.github.io/website/data/tp4_files.tgz

--2019-07-22 20:07:11--  https://cservan.github.io/website/data/tp4_files.tgz
Resolving cservan.github.io (cservan.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to cservan.github.io (cservan.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4517825 (4.3M) [application/octet-stream]
Saving to: ‘tp4_files.tgz’

tp4_files.tgz       100%[===================>]   4.31M  --.-KB/s    in 0.09s   

2019-07-22 20:07:12 (49.0 MB/s) - ‘tp4_files.tgz’ saved [4517825/4517825]



In [6]:

!rm data-l.tar.gz

rm: cannot remove 'data-l.tar.gz': No such file or directory


In [0]:
!tar xf  tp4_files.tgz

mv: target 'data_translation/ep7_tst.fr' is not a directory


In [0]:
!cat data_translation/EMEA_train.en >> train.en
!cat data_translation/ep7_train.en >> train.en
!cat data_translation/EMEA_train.fr >> train.fr
!cat data_translation/ep7_train.fr >> train.fr

# Subword Tokenization

We use Byte Pair Encoding for Subword Tokenization

https://www.aclweb.org/anthology/P16-1162

i => input<br>
o ==> Output(*.code)<br>
s ==> Symbol (Usually use 32000)<br>

learn_bpe ==> make code<br>
apply_bpe ==> apply subwordTokenization<br>

src-train, src-val,test ==> Need to apply src.code<br>
tgt-train,tgt-val ==> Need to apply tgt.code


In [0]:
!python OpenNMT-tf/third_party/learn_bpe.py -i train.fr -o fr.code -s 16000
!python OpenNMT-tf/third_party/learn_bpe.py -i train.en -o en.code -s 16000
!python OpenNMT-tf/third_party/apply_bpe.py -c fr.code -i train.fr -o train.fr.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c en.code -i train.en -o train.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c fr.code -i data_translation/EMEA_dev.fr -o EMEA_dev.fr.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c en.code -i data_translation/EMEA_dev.en -o EMEA_dev.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c fr.code -i data_translation/ep7_dev.fr -o ep7_dev.fr.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c en.code -i data_translation/ep7_dev.en -o ep7_dev.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c fr.code -i data_translation/EMEA_tst.fr -o EMEA_tst.fr.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c en.code -i data_translation/EMEA_tst.en -o EMEA_tst.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c fr.code -i data_translation/ep7_tst.fr -o ep7_tst.fr.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c en.code -i data_translation/ep7_tst.en -o ep7_tst.en.bpe

# Build Vocab

We will be working with some example data in toy-ende/ folder.
​
The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space:
​
1. src-train.txt
​
2. tgt-train.txt
​
3. src-val.txt
​
4. tgt-val.txt
​
​

Train data and validataion data are required for machine translation training.
​
Validation files are required and used to evaluate the convergence of the training. It usually contains no more than 5000 sentences.



In [10]:
!onmt-build-vocab --size 50000 --save_vocab vocab.fr train.fr.bpe
!onmt-build-vocab --size 50000 --save_vocab vocab.en train.en.bpe

W0722 20:10:09.476788 139918424053632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/decoders/rnn_decoder.py:435: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0722 20:10:10.589747 139918424053632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/adafactor.py:32: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 20:10:10.590256 139918424053632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/multistep_adam.py:36: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0722 20:10:15.957321 140192497944448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/decoders/rnn_decoder.py:435: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0722 20:10:1

In [11]:
!onmt-build-vocab --size 50000 --save_vocab toy-ende/src-vocab.txt toy-ende/src-train-bpe.txt

W0722 20:10:26.298670 140307859224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/decoders/rnn_decoder.py:435: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0722 20:10:27.140803 140307859224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/adafactor.py:32: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 20:10:27.141323 140307859224448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/multistep_adam.py:36: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

Traceback (most recent call last):
  File "/usr/local/bin/onmt-build-vocab", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/opennmt/bin/build_vocab.py", line 52, in main
    vocab.add_from_text(data_file, tokenizer=toke

In [0]:
!onmt-build-vocab --size 50000 --save_vocab toy-ende/tgt-vocab.txt toy-ende/tgt-train-bpe.txt

# Let's Make data.yml

```
model_dir: toy-ende/run/

data:
  train_features_file: toy-ende/src-train.txt
  train_labels_file: toy-ende/tgt-train.txt
  eval_features_file: toy-ende/src-val.txt
  eval_labels_file: toy-ende/tgt-val.txt
  source_words_vocabulary: toy-ende/src-vocab.txt
  target_words_vocabulary: toy-ende/tgt-vocab.txt

train:
  save_checkpoints_steps: 1000

  eval:
    eval_delay: 3600  # Every 1 hour
    external_evaluators: BLEU
infer:
    batch_size: 32


```










**Create a data.yml file on your computer and upload it to Google Colab.**

In [14]:
from google.colab import files
files.upload()

Saving data.yml to data.yml


{'data.yml': b"model_dir: 'gdrive/My Drive/model/run/'\n\ndata:\n  train_features_file: train.fr.bpe\n  train_labels_file: train.en.bpe\n  eval_features_file: ep7_dev.fr.bpe\n  eval_labels_file: ep7_dev.en.bpe\n  source_words_vocabulary: vocab.fr\n  target_words_vocabulary: vocab.en\n\ntrain:\n  batch_size: 1024\n  save_checkpoints_steps: 1000\n  maximum_features_length: 50\n  maximum_labels_length: 50\n\n  eval:\n    eval_delay: 3600  # Every 1 hour\n    external_evaluators: BLEU\ninfer:\n    batch_size: 32"}

# **Train the data(Basic)**

This command will start the training and evaluation loop of a small RNN-based sequence to sequence model.

If you want to use GPU , try add  below command (example use 1 GPU)
>  --num_gpus 1

Let's Check Available GPU


In [0]:
!nvidia-smi

Sun Jul 21 19:35:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

**Let's Train**

Model is locate in toy-ende/run

In [0]:
import tensorflow as tf
config = tf.ConfigProto(device_count = {'GPU': 1})
tf.Session(config=config)

In [0]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.list_devices())

W0721 19:05:27.706065 140174334187392 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 14129114277990025488), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4892493333467479829), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2641036094187648022), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 11326691738, 4360997152523710507)]


In [12]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

['/device:GPU:0']

In [0]:
!rm -r run

# **Train the data(Transformer)**

https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


> If you get GPU-related errors, try halving batch_size


***Just Change model_type***

Available Model ==> http://opennmt.net/OpenNMT-tf/package/opennmt.models.catalog.html


In [0]:
!onmt-main train_and_eval --model_type NMTSmall --auto_config --config data.yml --num_gpus 1

W0722 20:19:59.463116 140047122683776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/decoders/rnn_decoder.py:435: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0722 20:20:00.304202 140047122683776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/adafactor.py:32: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 20:20:00.304729 140047122683776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/multistep_adam.py:36: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0722 20:20:00.312719 140047122683776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/bin/main.py:111: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0722 20:20:00.312883

# **Translate**

Now that you have your model, you can start translating.



Output predictions into pred.txt

Translate Using desired model

--checkpoint_path run/baseline-enfr/avg/model.ckpt-200000

In [0]:
!onmt-main infer --auto_config    --config data.yml      --features_file toy-ende/src-test.txt --predictions_file toy-ende/pred.txt

W0721 19:30:37.586095 139732698646400 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0721 19:30:37.626138 139732698646400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/decoders/rnn_decoder.py:435: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0721 19:30:38.613473 139732698646400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/adafactor.py:32: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 19:30:38.614068 139732698646400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/optimizers/multistep_adam.py:36: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0721 19:30:38.622879 139732698646400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/opennmt/bin/main.py:11

# Translate to your chosen model

Add

--checkpoint_path toy-ende/run/model.ckpt-YOUR_MODEL

In [0]:
!onmt-main infer --auto_config    --config data.yml      --features_file toy-ende/src-test.txt --predictions_file toy-ende/pred.txt --checkpoint_path toy-ende/run/model.ckpt-YOUR_MODEL

# Detokenization

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.

We Use "sed" for BPE Detokenization


In [0]:
sed -i "s/@@ //g"  toy-ende/pred.txt

# Evaluation Using BLEU

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [0]:
perl  OpenNMT-tf/third_party/multi-bleu.perl toy-ende/ref.txt < toy-ende/pred.txt

If you have Any Question Please Email to  "bcj1210@naver.com"